# LSTUR: Analysis of Hyperparameter Optimization Results
This notebook provides analysis of the results obtained from the hyperparameter optimization for the LSTUR model, focusing on the effectiveness of different parameter configurations. The primary goal is to visualize and quantify the impact of each hyperparameter on the model's performance metrics.

LSTUR \[2\] is a news recommendation approach that captures users' both long-term preferences and short-term interests. The core of LSTUR is composed of a news encoder and a user encoder. The news encoder learns representations of news from their titles, while the user encoder learns long-term user representations from the embeddings of their IDs and short-term user representations from their recently browsed news via a GRU network.

## Properties of LSTUR:

- **Dual User Representations**: LSTUR captures both short-term and long-term preferences by using embeddings of users' IDs for long-term user representations and a GRU network to learn short-term user representations from recently browsed articles.
- **News Encoder**: Utilizes the news titles to generate news representations.
- **User Encoder**: Combines long-term and short-term user representations. Two methods are proposed for this combination:
  - Initializing the hidden state of the GRU network with the long-term user representation.
  - Concatenating both long-term and short-term user representations to form a unified user vector.
 
\[1\] https://github.com/ebanalyse/ebnerd-benchmark

\[2\] https://aclanthology.org/P19-1033/

In [ ]:
import os
import itertools
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

## Hyperparameter Grid Definition
This cell initializes the `param_grid` dictionary containing potential values for different hyperparameters of the LSTUR model. It then generates a list of all possible combinations of these parameters using a Cartesian product. This setup is essential for the subsequent steps of hyperparameter optimization, where each combination will be evaluated.

In [ ]:
param_grid = {
    'history_size': [10, 50, 100],
    'n_users': [20000, 50000, 70000],
    'title_size': [10, 50, 100],
    'learning_rate': [0.0001, 0.001, 0.01],
    'dropout':  [0.1, 0.3, 0.5]
}

combinations = list(itertools.product(
    param_grid['history_size'],
    param_grid['n_users'],
    param_grid['title_size'],
    param_grid['learning_rate'],
    param_grid['dropout']
))

## Model Evaluation Results Processing
This cell iterates through all combinations of hyperparameters generated previously to collect the evaluation results.

In [ ]:
results = []

for history_size, n_users, title_size, learning_rate, dropout in combinations:
    MODEL_NAME = f"LSTUR_h{history_size}_u{n_users}_t{title_size}_l{learning_rate}_d{dropout}"
    RESULTS_DIR = f"../output/evaluations/{MODEL_NAME}"
    results_file = os.path.join(RESULTS_DIR, 'evaluation_results.txt')
    
    if os.path.exists(results_file):
        with open(results_file, 'r') as file:
            # Read the results from the file
            eval_results = {}
            for line in file:
                key, value = line.strip().split(': ')
                eval_results[key] = float(value)
            
            # Add the hyperparameters to the results
            eval_results['history_size'] = history_size
            eval_results['n_users'] = n_users
            eval_results['title_size'] = title_size
            eval_results['learning_rate'] = learning_rate
            eval_results['dropout'] = dropout
            
            
            # Append to the results list
            results.append(eval_results)
    else:
        print(f"Results file not found for combination: history_size={history_size}, n_users={n_users}, title_size={title_size}")

# Visualization and Analysis of Model Performance Metrics

The following cells contain code for visualizing and analyzing the model performance based on the collected hyperparameter evaluation results. We use heatmaps to provide a clear visual representation of how different hyperparameter combinations and specific parameters like dropout rates affect model accuracy metrics.

In [ ]:
df = pd.DataFrame(results)
df

In [ ]:
# df['combination'] = df.apply(lambda row: f"h{row['history_size']}_u{row['n_users']}_t{row['title_size']}", axis=1)
df['combination'] = df.apply(lambda row: f"l{row['learning_rate']}_d{row['dropout']}", axis=1)

# Select only the relevant columns for the heatmap
heatmap_data = df[['combination', 'auc', 'mrr', 'ndcg@5', 'ndcg@10']]

# Set the combination column as the index
heatmap_data.set_index('combination', inplace=True)

# Normalize each metric individually
scaler = MinMaxScaler()
heatmap_data[['auc', 'mrr', 'ndcg@5', 'ndcg@10']] = scaler.fit_transform(heatmap_data[['auc', 'mrr', 'ndcg@5', 'ndcg@10']])

# Plot the heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(heatmap_data, annot=True, fmt=".4f", cbar=True)
plt.title('Heatmap of Accuracy Metrics for Different Hyperparameter Combinations')
plt.ylabel('Hyperparameter Combinations')
plt.xlabel('Accuracy Metrics')
plt.show()

In [ ]:
# Group by dropout and calculate the mean of each metric
mean_results = df.groupby('dropout').mean().reset_index()

# Select only the relevant columns for the heatmap
heatmap_data = mean_results[['dropout', 'auc', 'mrr', 'ndcg@5', 'ndcg@10']]

# Set the dropout column as the index
heatmap_data.set_index('dropout', inplace=True)

# Normalize each metric individually
scaler = MinMaxScaler()
heatmap_data[['auc', 'mrr', 'ndcg@5', 'ndcg@10']] = scaler.fit_transform(heatmap_data[['auc', 'mrr', 'ndcg@5', 'ndcg@10']])

# Plot the heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(heatmap_data, annot=True, fmt=".4f", cmap="YlGnBu", cbar=True)
plt.title('Heatmap of Normalized Accuracy Metrics for Different Dropout Rates')
plt.ylabel('Dropout Rate')
plt.xlabel('Accuracy Metrics')
plt.show()